In [114]:
import pandas as pd
import numpy as np
import subprocess
import itertools
import time
from scipy.stats import spearmanr,pearsonr


In [4]:
targets = pd.read_csv("targets.csv", header=None)
lstm_predicts = pd.read_csv("lstm_predicts.csv", header=None)
lstm_predicts_teacher = pd.read_csv("lstm_predicts_teacher.csv", header=None)
unigram_predicts = pd.read_csv("unigram_predicts.csv", header=None)

In [20]:
target_arr = np.array(targets.iloc[:, 0])
lstm_arr = np.array(lstm_predicts.iloc[:, 0])
lstm_teacher_arr = np.array(lstm_predicts_teacher.iloc[:, 0])
unigram_arr = np.array(unigram_predicts.iloc[:, 0])

In [40]:
target_mfes = []
lstm_mfes = []
lstm_teacher_mfes = []
unigram_mfes = []

In [105]:
def calculate_MFEs(arr):
    mfes = []
    seqs = []
    for i in range(len(arr)):
        seq = arr[i]
        if (len(seq) > 120):
            seq = seq[:120]
        text_file = open("input.txt", "w")
        text_file.write(seq)
        text_file.close()
        seqs.append(seq)
#         print ("reached1")
        process = subprocess.Popen(["RNAFold", "input.txt"], stdout=subprocess.PIPE)
#         print ("reached2")
        out, err = process.communicate()
        out = out.decode('utf8')
        mfes.append(float(out[out.rfind("(")+1: out.rfind(")")]))
    return mfes

In [106]:
calculate_MFEs(target_arr[:2])

[-6.8, -1.5]

In [109]:
target_mfes = calculate_MFEs(target_arr)
df = pd.DataFrame(target_mfes, columns=["MFE"])
df.to_csv("target_mfes.csv", index=False, header=False)

In [110]:
lstm_mfes = calculate_MFEs(lstm_arr)
df = pd.DataFrame(lstm_mfes, columns=["MFE"])
df.to_csv("lstm_mfes.csv", index=False, header=False)

In [111]:
lstm_mfes_teacher = calculate_MFEs(lstm_teacher_arr)
df = pd.DataFrame(lstm_mfes_teacher, columns=["MFE"])
df.to_csv("lstm_mfes_teacher.csv", index=False, header=False)

In [112]:
unigram_mfes = calculate_MFEs(unigram_arr)
df = pd.DataFrame(unigram_mfes, columns=["MFE"])
df.to_csv("unigram_mfes.csv", index=False, header=False)

In [116]:
spearmanr(target_mfes, lstm_mfes)

SpearmanrResult(correlation=0.579647617306106, pvalue=5.583245088930676e-119)

In [117]:
spearmanr(target_mfes, lstm_mfes_teacher)

SpearmanrResult(correlation=0.6580908815301526, pvalue=3.853321995850281e-164)

In [118]:
spearmanr(target_mfes, unigram_mfes)

SpearmanrResult(correlation=0.5732090176188556, pvalue=8.388161004982853e-116)